<a href="https://colab.research.google.com/github/jumairamiller/US-Wildfire-Data-Visualisation-/blob/main/VisulaizationCoursework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# To get authorization code, follow the link and log into gmail account:

username - **jummyller.m27@gmail.com**

password - **Arch@mb@ultM27**

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# required imports
import numpy as np   
import pandas as pd
import altair as alt
from altair import datum, expr
import matplotlib.pyplot as plt
import sklearn
import math
from math import e
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score 


In [ ]:
#Load Data
states_data = pd.read_csv('drive/My Drive/states.csv')
df=pd.read_csv('drive/My Drive/firesData.csv')
df

FileNotFoundError: ignored

In [ ]:
df.columns

In [ ]:
"""
Note to the dev. The variable "Decade" was chosen haphazardly and represents
a now-defunct design choice. A total refactor to instead use TIME_BIN or
TIME_PERIOD would aid readability.
"""

def to_decade(year):
  if year <= 1999:
    return "1993-1999"
  elif year <= 2004:
    return "2000-2004"
  elif year <= 2009:
    return "2005-2009"
  elif year <= 2015:
    return "2010-2015"
  return None

# Create the Decade column for time series binning
df["DECADE"] = df["FIRE_YEAR"].apply(to_decade)
df["DECADE_NUMBER"] = df["DECADE"].apply(lambda d: ["1993-1999", "2000-2004", "2005-2009", "2010-2015"].index(d))

df.sample(n=10)

In [ ]:
# Frequency of fire, by state and decade
df["STATE_DECADE_FREQUENCY"] = df.groupby(["DECADE", "STATE"])["STATE"].transform(pd.Series.count)
# Precalculate the sum of fire size by state and decade
df["STATE_DECADE_FIRE_SIZE_SUM"] = df.groupby(["DECADE", "STATE"])["FIRE_SIZE"].transform(pd.Series.sum)

df


In [ ]:
"""
Note: The copious use of df.apply(...) makes this section run particularly slowly
especially on the full datatset.
Less heavy data transformation could be used to reduce this overhead.
"""

def grade_fire_size(fire_size):
  """
  Takes fire size (in acres) and returns the fire size designation as an index.
  These values are taken from https://www.nwcg.gov/data-standards/approved/fire-size-class
  As the dataset tracks all fires >= 5000 as a G we cutoff at this place too
  These can be converted to letters alphabetically
  """
  size_cutoffs = [0.26, 10, 100, 300, 1000, 5000]
  indicies = [1, 2, 3, 4, 5, 6]
  for index, limit in zip(indicies, size_cutoffs):
    if fire_size < limit:
      return index
  # Code G is for all fires larger than 5000 acres
  return 7

def weight_function(x):
  """
  How the weight of a value should be decided
  params:
    x: ndarray or df projection
  """
  return x

# Precalc weights
df["WEIGHTS"] = weight_function(df["FIRE_SIZE"])

aggs = df.groupby(["STATE", "DECADE"])["FIRE_SIZE"].agg(
    MIN_FIRE_SIZE_STATE_DECADE=np.min,
    MAX_FIRE_SIZE_STATE_DECADE=np.max,
    MEAN_FIRE_SIZE_STATE_DECADE=np.mean).reset_index()
aggs = pd.merge(aggs, df.groupby(["STATE", "DECADE"]).apply(lambda x: np.average(x['FIRE_SIZE'])).reset_index(name="WAV_FIRE_SIZE_STATE_DECADE"), on=['STATE', 'DECADE'])

df = pd.merge(df, aggs, on=['STATE', 'DECADE'])

df["CATAGORY"] = df.apply(lambda row: grade_fire_size(row.WAV_FIRE_SIZE_STATE_DECADE), axis = 1)
df["CATAGORY_LETTER"] = df.apply(lambda row: chr(ord('A') + (row.CATAGORY - 1)), axis = 1)
df["FIRE_SIZE_CLASS"].value_counts().sort_index().plot.bar()

# For distribution mapping purposes
plt.show()
df["CATAGORY"].value_counts().sort_index().plot.bar()
plt.show()

x_max = 10
plt.plot(np.linspace(0, x_max, 10000), weight_function(np.linspace(0, x_max, 10000)))
plt.plot([0, x_max], [1, 1], color="red")
plt.show()

df.sample(n=10)

In [ ]:
# Linear Regression model shorthand

def train_model(Xs, ys):
  X_train, X_test, y_train, y_test = train_test_split(Xs, ys, test_size=0.3, random_state=0)
  model = LinearRegression()
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)

  return model, r2_score(y_test, y_pred)

def plot_model(Xs, ys, model):
  plt.scatter(Xs, ys)
  plt.plot((min(Xs), max(Xs)), (model.predict([[min(Xs)]])[0][0], model.predict([[max(Xs)]])[0][0]), 'red')
  plt.show()

In [ ]:
state_models = {}
for state in df["STATE"].unique():
  state_proj = df[df["STATE"] == state]
  if(len(state_proj) <= 2):
    continue
  state_models[state] = {}
  freqs = state_proj.groupby(["FIRE_YEAR"])["FIRE_YEAR"].count().reset_index(name="FREQUENCY")
  # Add the training data to each model
  state_models[state]["Fire Size"] = train_model(state_proj["FIRE_YEAR"].values.reshape(-1,1), state_proj["FIRE_SIZE"].values.reshape(-1,1))
  state_models[state]["Fire Freq"] = train_model(freqs["FIRE_YEAR"].values.reshape(-1,1), freqs["FREQUENCY"].values.reshape(-1,1))
  state_models[state]["Fire Catagory"] = train_model(state_proj["DECADE_NUMBER"].values.reshape(-1,1), state_proj["CATAGORY"].values.reshape(-1,1))
print(state_models)

In [ ]:
# Show the states in state data on a chart, for debugging purposes
alt.Chart(states_data).mark_circle(size=60).encode(
    x='LON',
    y='LAT',
    tooltip=["STATE"]
).interactive()

In [ ]:
"""
Code for circle repulsion
"""
class Circle:
	def __init__(self, x, y, r, c = '#000000'):
		self.xy = np.array([x,y])
		self.r = r
		self.c = c

	@property
	def x(self):
		return self.xy[0]

	@property
	def y(self):
		return self.xy[1]

	def sqd_distance(self, other):
		delta_x = other.x - self.x
		delta_y = other.y - self.y
		return delta_x * delta_x + delta_y * delta_y

	def distance(self, other):
		return math.sqrt(self.sqd_distance(other))

	def direction(self, other):
		return other.xy - self.xy

	def intersects(self, other):
		r_comb = self.r + other.r
		# Bounding box test
		if self.x - r_comb < other.x < self.x + r_comb and self.y - r_comb < other.y < self.y + r_comb:
			return self.sqd_distance(other) < r_comb ** 2
		return False

	@staticmethod
	def random_circle(max_x, max_y, max_r):
		x = np.random.uniform(0, max_x)
		y = np.random.uniform(0, max_y)
		c = '#%02x%02x%02x' % (int(255*x/max_x), 0, int(255*y/max_y))
		return Circle(np.random.uniform(0, max_x), np.random.uniform(0, max_y), np.random.uniform(0, max_r), c=c)

	def __str__(self):
		return "CIRCLE ({0}, {1}) r={2}".format(self.x, self.y, self.r)

def find_center(circles):
	xs = [c.x for c in circles]
	ys = [c.y for c in circles]
	return np.mean(xs), np.mean(ys)

def circle_repel(circles):
	# Sort circles by distance from the center (assume that we wish to handle the furthest from center first
	midpoint = np.array(find_center(circles))
	midpoint_circle = Circle(midpoint[0], midpoint[1], 0)
	done = False
	tolerence = 0.25
	# Separate circles out
	while not done:
		done = True
		sorted_circles = sorted(circles, reverse=True, key=lambda c: midpoint_circle.sqd_distance(c))

		for i in range(len(sorted_circles) - 1):
			circle = sorted_circles[i]
			for j in range(i+1, len(sorted_circles)):
				other_circle = sorted_circles[j]
				if circle != other_circle and circle.intersects(other_circle):
					# In the case that the two circles lie on the same point exactly
					if all(circle.xy == other_circle.xy):
						# In the case that this is also the exact midpoint
						if all(circle.xy == midpoint):
							travel_vector = np.random.random(2)
						else:
							travel_vector = circle.xy - midpoint
					else:
						travel_vector = other_circle.direction(circle)

					travel_vector = travel_vector / np.linalg.norm(travel_vector)
					# By how much do the circles need to move
					overlap = ((circle.r + other_circle.r) - circle.distance(other_circle)) + tolerence

					ratio = other_circle.r / (circle.r + other_circle.r)
					this_move = ratio * overlap
					other_move = (1 - ratio) * overlap
					circle.xy = circle.xy + (travel_vector * this_move)
					other_circle.xy = other_circle.xy - (travel_vector * other_move)
					done = False

	# Collect them together
	sorted_circles = sorted(circles, reverse=False, key=lambda c: midpoint_circle.distance(c) - c.r)
	for circle in sorted_circles:
		midpoint_distance = midpoint_circle.distance(circle) - circle.r
		closest_neighbour_distance = None
		closest_neighbour = None
		for other_circle in sorted_circles:
			if circle != other_circle and midpoint_circle.distance(other_circle) - circle.r <= midpoint_distance:
				d = circle.distance(other_circle) - (circle.r + other_circle.r)
				if closest_neighbour is None or d < closest_neighbour_distance:
					closest_neighbour = other_circle
					closest_neighbour_distance = d
		if closest_neighbour is None:
			#handle first circle case
			continue
		if closest_neighbour_distance < (tolerence * 2):
			# Already borders another circle
			continue
		travel_vector = circle.direction(closest_neighbour)
		travel_vector = (closest_neighbour_distance * travel_vector) / np.linalg.norm(travel_vector)
		circle.xy = circle.xy + travel_vector



def plot_circles(circles, midpoint=None):
	xs = [c.x for c in circles]
	ys = [c.y for c in circles]
	rs = [c.r for c in circles]
	xs_range = max(xs) - min(xs)
	ys_range = max(ys) - min(ys)
	max_range = max(xs_range, ys_range) + (2 * max(rs))
	xs_midpoint = min(xs) + (xs_range / 2)
	ys_midpoint = min(ys) + (ys_range / 2)

	figure, ax = plt.subplots()
	plt.xlim(xs_midpoint - max_range / 2, xs_midpoint + max_range / 2)
	plt.ylim(ys_midpoint - max_range / 2, ys_midpoint + max_range / 2)

	for circle in circles:
		draw_circle = plt.Circle((circle.x, circle.y), circle.r, fill=False, color=circle.c)
		ax.add_artist(draw_circle)

	if midpoint is not None:
		plt.plot(midpoint[0], midpoint[1], 'b+')

	ax.set_aspect(1)
	plt.show()
 
def pack_circles(data, x_key, y_key, size_key, foreign_key=None, fit_unit_circle=True, size_normalizing_factor = (.05, .25)):
  if foreign_key is not None:
    circle_df = data[[x_key, y_key, size_key, foreign_key]].copy()
  else:
    circle_df = data[[x_key, y_key, size_key]].copy()

  x_range = circle_df[x_key].max() - circle_df[x_key].min()
  y_range = circle_df[y_key].max() - circle_df[y_key].min()
  size_range = circle_df[size_key].max() - circle_df[size_key].min()
  max_range = max(x_range, y_range)
  # Set all sizes between one tenth and one quarter of the screen. This is set experimentally
  scale = max_range * size_normalizing_factor[0], max_range * size_normalizing_factor[1]
  if size_range == 0:
    circle_df[size_key] = 1
  else:
    circle_df[size_key] = (circle_df[size_key] - circle_df[size_key].min()) / size_range
    circle_df[size_key] *= scale[1] - scale[0]
    circle_df[size_key] += scale[0]
  
  circle_data = {
    i if foreign_key is None else row[foreign_key]: Circle(row[x_key], row[y_key], row[size_key]) for i, row in circle_df.iterrows()
  }
  circles = list(circle_data.values())
  circle_repel(circles)
  if foreign_key is None:
    packed_data = pd.DataFrame({
      'CIRCLE_X': [c.x for c in circles],
      'CIRCLE_Y': [c.y for c in circles],
      'CIRCLE_R': [c.r for c in circles]
      })
  else:
    packed_data = pd.DataFrame({
    'CIRCLE_X': [c.x for c in circles],
    'CIRCLE_Y': [c.y for c in circles],
    'CIRCLE_R': [c.r for c in circles],
    foreign_key: [row[foreign_key] for _, row in circle_df.iterrows()]
    })
  if not fit_unit_circle:
    return packed_data
  # Transform the data into altair chart space. Altair, it seems,
  # DOES NOT WANT ANYBODY plotting a circle on their charts the easy way
  # so this is the workaround.
  extents_x_min = (packed_data["CIRCLE_X"] - packed_data["CIRCLE_R"]).min()
  extents_x_max = (packed_data["CIRCLE_X"] + packed_data["CIRCLE_R"]).max()
  extents_y_min = (packed_data["CIRCLE_Y"] - packed_data["CIRCLE_R"]).min()
  extents_y_max = (packed_data["CIRCLE_Y"] + packed_data["CIRCLE_R"]).max()
  midpoint_x = extents_x_min + ((extents_x_max - extents_x_min) / 2)
  midpoint_y = extents_y_min + ((extents_y_max - extents_y_min) / 2)
  extents_max = max(extents_x_max - extents_x_min, extents_y_max - extents_y_min)
  # Transform the data into altair space (normalized between -1 and 1)
  packed_data["CIRCLE_X"] = packed_data["CIRCLE_X"] - midpoint_x
  packed_data["CIRCLE_Y"] = packed_data["CIRCLE_Y"] - midpoint_y
  packed_data["CIRCLE_X"] = packed_data["CIRCLE_X"] / (extents_max * 0.5)
  packed_data["CIRCLE_Y"] = packed_data["CIRCLE_Y"] / (extents_max * 0.5)
  packed_data["CIRCLE_R"] = packed_data["CIRCLE_R"] / (extents_max * 0.5)
  packed_data["CIRCLE_A"] = (packed_data["CIRCLE_R"] * 2) ** 2
  return packed_data

# Used to test repulsion visually
circle_data = {i: Circle(row["LON"], row["LAT"], np.random.uniform(8, 20)) for i, row in states_data.iterrows()}
circles = list(circle_data.values())
plot_circles(circles, find_center(circles))
circle_repel(circles)
plot_circles(circles, find_center(circles))


In [ ]:
# Combine state and wildfire data so state names are kept.
df = pd.merge(df,
         states_data,
         left_on="STATE",
         right_on="ABBV",
         suffixes=("_ABBV", None),
         how='left')



In [ ]:
## Aggregate Dataframe
working_data = df.groupby(["DECADE", "STATE"]).first().reset_index()
working_data = working_data.drop(["OBJECTID", 'Unnamed: 0', 'OBJECTID', 'FOD_ID', 'FPA_ID', 'SOURCE_SYSTEM_TYPE', 'SOURCE_SYSTEM', 'NWCG_REPORTING_AGENCY', 'NWCG_REPORTING_UNIT_ID', 'NWCG_REPORTING_UNIT_NAME', 'SOURCE_REPORTING_UNIT', 'SOURCE_REPORTING_UNIT_NAME', 'LOCAL_FIRE_REPORT_ID',
       'LOCAL_INCIDENT_ID', 'FIRE_CODE', 'FIRE_NAME', 'ICS_209_INCIDENT_NUMBER', 'ICS_209_NAME', 'MTBS_ID', 'MTBS_FIRE_NAME', 'COMPLEX_NAME', 'DISCOVERY_DATE', 'DISCOVERY_DOY', 'DISCOVERY_TIME', 'STAT_CAUSE_CODE', 'CONT_DATE', 'CONT_DOY', 'CONT_TIME', 'LATITUDE',
       'LONGITUDE', 'OWNER_CODE', 'OWNER_DESCR', 'COUNTY', 'FIPS_CODE', 'FIPS_NAME', 'Shape', 'WEIGHTS', "STAT_CAUSE_DESCR", "FIRE_SIZE", "FIRE_SIZE_CLASS", "FIRE_YEAR"], axis=1)
for state in working_data["ABBV"].unique():
  if state not in state_models.keys():
    continue
  size_model, _ = state_models[state]["Fire Size"]
  freq_model, _ = state_models[state]["Fire Freq"]
  cat_model, _ = state_models[state]["Fire Catagory"]
  old_row = working_data.loc[working_data['ABBV'] == state]
  catagory = int(round(min(max(cat_model.predict([[2021]])[0][0], 1), 7)))
  freq =  max(0, freq_model.predict([[2021]])[0][0])
  if freq == 0:
    continue
  # Add synthetic data
  new_row = pd.DataFrame({
      "DECADE": ["2021 Predictions"],
      "STATE": [old_row["STATE"].iloc[0]],
      "STATE_ABBV": [old_row["STATE_ABBV"].iloc[0]],
      "DECADE_NUMBER": [-1],
      "STATE_DECADE_FREQUENCY":[ freq],
      "STATE_DECADE_FIRE_SIZE_SUM": [None],
      "MIN_FIRE_SIZE_STATE_DECADE": [None],
      "MAX_FIRE_SIZE_STATE_DECADE": [None],
      "MEAN_FIRE_SIZE_STATE_DECADE": [None],
      "WAV_FIRE_SIZE_STATE_DECADE": [None],
      "CATAGORY": [catagory],
      "CATAGORY_LETTER": [chr(ord('A') + (catagory - 1))],
      "ABBV": [old_row["ABBV"].iloc[0]],
      "LAT": [old_row["LAT"].iloc[0]],
      "LON": [old_row["LON"].iloc[0]],
      "POP": [None]
  })
  working_data = working_data.append(new_row, ignore_index=True)
working_data

In [ ]:
acc_list = []
for key in state_models.keys():
  for model_key in state_models[key].keys():
    acc_list.append(state_models[key][model_key][1])
acc_list = sorted(acc_list)
acc_list = acc_list[5:-5]
plt.boxplot(acc_list)
print(acc_list)

In [ ]:
cause_counts = df.groupby(["STATE", "DECADE", "STAT_CAUSE_DESCR"])["STAT_CAUSE_DESCR"].count().reset_index(name="COUNT")
sizes_counts = df.groupby(["STATE", "DECADE", "FIRE_SIZE_CLASS"])["FIRE_SIZE_CLASS"].count().reset_index(name="COUNT")
sizes_counts = pd.merge(sizes_counts, df.groupby(["STATE", "DECADE"])["DECADE"].count().reset_index(name="TOTAL"), on=["STATE", "DECADE"])
sizes_counts

In [ ]:
chart_size = 600

def get_circle_positions_by_decade(raw_data, decade):
  decade_data = raw_data[raw_data["DECADE"] == decade].groupby(["ABBV"])[("ABBV", "LAT", "LON", "STATE_DECADE_FREQUENCY")].mean()
  decade_data = decade_data.reset_index()
  return pack_circles(decade_data, "LON", "LAT", "STATE_DECADE_FREQUENCY", "ABBV")



decades = list(working_data["DECADE"].unique())
circle_positions = None
for decade in decades:
  d = get_circle_positions_by_decade(working_data, decade)
  d["DECADE"] = decade
  d["MAX_PIXEL_AREA"] = (d['CIRCLE_R'].max() * chart_size) ** 2
  if circle_positions is None:
    circle_positions = d
  else:
    circle_positions = circle_positions.append(d, ignore_index=True)

working_data = pd.merge(working_data, circle_positions, on=["ABBV", "DECADE"])


In [ ]:
def quartile(n):
    def quartile_(x):
        return np.percentile(x, n * 25)
    quartile_.__name__ = 'q%s' % n
    return quartile_

stats = df.groupby(["STATE", "DECADE"]).agg({"FIRE_SIZE": [np.min, np.max, np.mean, quartile(1), quartile(3)]}).reset_index()
stats.columns = [col[0] if len(col[-1]) == 0 else col[-1] for col in stats.columns.values]
stats

In [ ]:
def hex_to_rgb(hex):
  return tuple(int(hex.lstrip('#')[i:i+2], 16) for i in (0, 2, 4))

def rgb_to_hex(rgb):
  return '#%02x%02x%02x' % rgb

def interpolate_hex_codes(start, end, t):
  return rgb_to_hex(interpolate_rgb(hex_to_rgb(start), hex_to_rgb(end), t))

def interpolate_rgb(rgb1, rgb2, t):
  return tuple([interpolate_int(x, y, t) for x, y in zip(rgb1, rgb2)])

def interpolate_int(i1, i2, t):
  return round(i1 + ((i2 - i1) * t))
  

In [ ]:
color_range = "#ffffb2", "#b10026"
mid_color = interpolate_hex_codes(color_range[0], color_range[1], 0.5)

"""
Definition for Altair Representation of Area Schematized Cartogram
"""

decade_radio = alt.binding_radio(options=decades)
decade_select = alt.selection_single(fields=['DECADE'], bind=decade_radio, name="Decade Select", init={'DECADE': decades[0]})

state_selector = alt.selection_multi(fields=['STATE'])

""" CREATE AREA CARTOGRAM """

"""
For the absolute life of me I can't figure out if altair will allow dynamic ranges to be set
there is absolutely no support for plotting circles with a specificed size
this is the closest I have been able to get
"""

base = alt.Chart(working_data).encode(
  x = alt.X('CIRCLE_X', scale = alt.Scale(domain=[-1, 1]), axis=None),
  y = alt.Y('CIRCLE_Y', scale = alt.Scale(domain=[-1, 1]), axis=None),
).transform_filter(
    decade_select
)

tooltips = [
            alt.Tooltip('STATE', title="State"),
            alt.Tooltip('ABBV', title="Code"),
            alt.Tooltip('STATE_DECADE_FREQUENCY', title="Frequency"),
            alt.Tooltip('CATAGORY_LETTER', title="Weighted Fire Class"),
            ]

chart = base.mark_circle(stroke='black', strokeWidth=2).encode(
  color = alt.condition(state_selector, 'CATAGORY:Q', alt.value('lightgray'), legend=None, scale=alt.Scale(range=color_range)),
  size= alt.Size('CIRCLE_A', scale=alt.Scale(domain=[0, 4], range=[0, chart_size * chart_size]), legend=None),
  tooltip=tooltips
).properties(width=chart_size, height=chart_size)

text = base.mark_text().encode(
    text='ABBV',
    tooltip=tooltips
)
labeled_chart = ((chart + text)).add_selection(
      state_selector
).add_selection(
    decade_select
)

legend = alt.Chart(working_data).mark_circle(size=150).encode(
      x=alt.X(
          "CATAGORY_LETTER:N",
          axis=alt.Axis(domain=False, ticks=False, labelAngle=0, orient='top'),
          title="Weighted average fire size classification"
      ),

      color=alt.Color('CATAGORY:Q', legend=None, scale=alt.Scale(range=("#ffff00", "#ff0000")))
)

wav_bars = alt.Chart(working_data).mark_bar().encode(
    x=alt.X("count(CATAGORY_LETTER)", axis=alt.Axis(title='Overall Occurences')),
    y=alt.Y("CATAGORY_LETTER:N", axis=alt.Axis(title='Classification')),
    color=alt.Color('CATAGORY_LETTER', scale=alt.Scale(range=[mid_color, mid_color]), legend=None),
    tooltip=[alt.Tooltip('CATAGORY_LETTER', title="Class"), alt.Tooltip('count(CATAGORY_LETTER)', title="#")]
).transform_filter(
    decade_select
)

causes_count = len(cause_counts["STAT_CAUSE_DESCR"].unique())
causes_selection = alt.selection_multi(fields=['STAT_CAUSE_DESCR'])

causes_bars = alt.Chart(cause_counts).mark_bar().encode(
    x='STATE',
    y=alt.Y('COUNT'),
    color=alt.Color('STAT_CAUSE_DESCR', legend=None),
    tooltip=[alt.Tooltip('STATE', title="State"), alt.Tooltip('COUNT', title="#"), alt.Tooltip('STAT_CAUSE_DESCR', title="Cause")]
).transform_filter(
    decade_select
).transform_filter(
    state_selector
).transform_filter(
    causes_selection
).add_selection(
    state_selector
).add_selection(
    causes_selection
)

causes_legend = alt.Chart(cause_counts).mark_circle().encode(
    y=alt.Y('STAT_CAUSE_DESCR:N', axis=alt.Axis(orient='right')),
    color=alt.condition(causes_selection, alt.Color('STAT_CAUSE_DESCR:N', legend=None), alt.value('lightgray'))
).add_selection(
    causes_selection
)
causes_bars = causes_bars | causes_legend


fire_classifications_count = len(sizes_counts["FIRE_SIZE_CLASS"].unique())
sizes_bars = alt.Chart(sizes_counts).mark_bar().encode(
    x='STATE',
    y='COUNT',
    color=alt.Color('FIRE_SIZE_CLASS:O', scale=alt.Scale(range=[interpolate_hex_codes(color_range[0], color_range[1], i/(fire_classifications_count-1)) for i in range(fire_classifications_count)], type="ordinal")),
    tooltip=[alt.Tooltip('STATE', title="State"), alt.Tooltip('TOTAL', title="Total"),  alt.Tooltip('COUNT', title="#"), alt.Tooltip('FIRE_SIZE_CLASS', title="Class")]
).transform_filter(
    decade_select
).transform_filter(
    state_selector
).add_selection(
    state_selector
)

outlines = alt.Chart(sizes_counts).mark_bar(filled=False, color='black', strokeWidth=0.5).encode(
    x='STATE',
    y='TOTAL',
    #tooltip=[alt.Tooltip('STATE', title="State"), alt.Tooltip('TOTAL', title="Total"),  alt.Tooltip('COUNT', title="#"), alt.Tooltip('FIRE_SIZE_CLASS', title="Class")]
).transform_filter(
    decade_select
).transform_filter(
    state_selector
)

sizes_bars = outlines + sizes_bars

alt.data_transformers.disable_max_rows()
whiskers = alt.LayerChart(stats).encode(
    x='STATE',
    tooltip=['amin:Q', 'q1:Q', 'mean:Q', 'q3:Q', 'amax:Q']
).add_layers(
    alt.Chart().mark_rule().encode(y=alt.Y('amin:Q', scale=alt.Scale(type='log')), y2='amax:Q'),
    alt.Chart().mark_bar(width=15).encode(y='q1:Q', y2='q3:Q'),
    alt.Chart().mark_tick(color='white', width=15).encode(y='mean:Q'),
).transform_filter(
    decade_select
).transform_filter(
    state_selector
)

((labeled_chart | (legend & wav_bars & whiskers)) & alt.hconcat(causes_bars, sizes_bars, resolve = alt.Resolve(scale=alt.LegendResolveMap(color=alt.ResolveMode('independent')))))


**TESTING GRAVEYARD**

In [ ]:
model, _ = train_model(df['FIRE_YEAR'].values.reshape(-1,1), df['FIRE_SIZE'].values.reshape(-1,1))
plot_model(df['FIRE_YEAR'], df['FIRE_SIZE'], model)

In [ ]:
X = df['FIRE_YEAR'].values.reshape(-1,1)
y = df['FIRE_SIZE'].values.reshape(-1,1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

model = LinearRegression()
model.fit(X_train, y_train)



In [ ]:
#Finding the intercept and slope of the data

print(model.intercept_)
print(model.coef_)

In [ ]:
y_pred = model.predict(X_test)

#Comparing with actual data

plottableDf = pd.DataFrame({'Actual': y_test.flatten(), 'Predicted': y_pred.flatten()})
plottableDf

In [ ]:
df1 = plottableDf.head(20)
df1.plot(kind='bar',figsize=(16,10))
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.show()